In [20]:
from cdag import Graph, load_data, read_table, parse, tokenize
from personal import database_connection
import re, pandas as pd
from collections import defaultdict

In [2]:
graph = Graph()
load_data(graph)

In [3]:
# primary_groups: CHIP (CS only), PS (irrelevant), ZKH (tree root), plus some others
root = graph.find_one_node('Group', searchcode='ZKH')

In [4]:
for child in root.adjacent('haschild'):
    print(child)

In [5]:
# read HiX tables
# method 1: from CSV files
# method 2: from database
# connection = database_connection('hixacceptatie')
# iv = pd.read_sql(sql=f"select * from [hix_acc].[dbo].[config_instvars];", con=connection
iv = pd.read_csv('config_instvars.csv')
iv['VALUE'] = iv['VALUE'].fillna('X""')
# iv.groupby('INSTTYPE').count()

In [6]:
# read HiX tables (continued)
wc = pd.read_csv('config_workcontext.csv')
wc.set_index('Id', inplace=True)
ws = pd.read_csv('config_wcsegments.csv')
ws.set_index('ConfiguredWorkContextId', inplace=True)
wsa = ws.loc[ws['Disabled'] == False]
wcs = wc.join(wsa, how='inner')
gr = pd.read_csv('ziscon_groepen.csv')
#us = pd.read_csv('ziscon_user.csv',
#                 dtype={'FUNKTIE':'string', 'BEHEERDER':'boolean', 'WACHTWOORD':'string',
#                        'DISABLED': 'boolean', 'UZINR': 'string', 'ROLE': 'string', 'WRONGCHECKS': 'string'})

In [29]:
# BEH: doss_noodprexemp geel value=CM
#      doss_noodprsee   groen value=CT
#      ond_rechten/widocarc CSSSSSF 5xblauw 1xrood
# kijk ook bij Document WI_EPDDASHBOARD. inverted=true -> alles behalve
cids = re.compile('^C\w{6,10}(,\w{6,10})*$')
cx1  = re.compile('^C[A-Z]$')
cx6  = re.compile('^C[A-Z]{6}$')
lx1  = re.compile('^L[A-Z]$')
lx6  = re.compile('^L[A-Z]{6}$')
idx  = defaultdict(int)
for elt in iv.to_dict('records'):
    if elt['INSTTYPE'] in ('G', 'D', 'U'):
        continue
    value = elt['VALUE'].rstrip('\x01\u00a9')
    if 'if(' in value or len(value) == 1 or \
       'Value(' in value or 'CRLF' in value: continue
    if value.startswith('C') and '{' in value:
        idx['C{}'] += 1
    elif value.startswith('C') and 'xml' in value:
        idx['C xml'] += 1
    elif cids.search(value):
        idx['Cids'] += 1
    elif cx1.search(value):
        idx['Cx1'] += 1
    elif cx6.search(value):
        idx['Cx6'] += 1
    elif lx1.search(value):
        idx['Lx1'] += 1
    elif lx6.search(value):
        idx['Lx6'] += 1
    elif value.startswith('C') and value[1] in '0123456789':
        idx['Cd'] += 1
    elif value.startswith('N') and value[1] in '0123456789':
        idx['Nd'] += 1
    else:
        idx[value] += 1
count = 0
for k, v in idx.items():
    print(f"{k}: {v}")
    count += 1
    if count > 30: break

In [18]:
#x = iv.loc[(iv['OWNER'] == '@GZH0085') & iv['NAAM'].str.contains('AG_US')].to_dict('records')
#x[0]
iv.loc[(iv['OWNER'] == '@GZH0085')]

In [9]:
parse(x[0]['VALUE'])

In [16]:
# wcs.loc[(wcs['OwnerId']=='SPEANE') & wcs['SettingId'].str.contains('AG')]
wcs.loc[(wcs['OwnerId'] == 'ZH0085') & wcs['SettingId'].str.contains('AG')]

In [11]:
#occurring, missing = {}, set()
#sep = '-' * 10
#for k, elt in enumerate(type_c.to_dict('records')):
#    value = guid_rx.sub(resolve_guid, elt['VALUE'])
#    # matches = guid_rx.findall(value)
#    if k < 5:
#        print(f"{sep}\nvalue {k}")
#        tokens = tokenize(value)
#        obj = parse(tokens)
#        for elt in obj:
#            print('\t'+elt)